# 建立 AWS CLI 環境

In [1]:
from dotenv import load_dotenv
import os
import boto3

# 載入 .env 檔案中的環境變數
load_dotenv()

# 建立 STS 客戶端
sts_client = boto3.client("sts")

# 呼叫 get_caller_identity 以取得當前 AWS 帳號資訊
response = sts_client.get_caller_identity()
account_id = response['Account']

print(f"AWS Account ID: {account_id}")

AWS Account ID: 339712730744


Version: 02.14.2023

# Lab 7.1: Implementing a Multilingual Solution

本實驗將使用三種不同的 AWS 服務來建立將音訊檔案翻譯為不同語言的文字的解決方案。

## Introducing the business scenario

本實驗將在多種語言之間翻譯影片。

## Lab steps

1. Amazon Transcribe example
2. Amazon Translate example
3. Amazon Polly example
4. Challenge exercise

## 1. Amazon Transcribe example

1. 本部分使用 boto3  用戶端呼叫 Amazon Transcribe 將音訊檔案轉換為文字，運行範例後，可前往 `Amazon Transcribe` 控制台查看轉錄內容。

2. 轉錄需要幾分鐘才能完成。


In [7]:
# 用於生成全域唯一識別碼（UUID）
import uuid
# 用於處理 JSON 格式的數據
import json
# 這是 AWS 的 Python SDK，用於與 AWS 服務互動
import boto3
# 從 time 模組匯入 sleep 函數，用於讓程式暫停指定的秒數
from time import sleep

# 定義 S3 bucket 的名稱，這裡使用的是 AWS S3 存儲資源的名稱 
bucket = 'c133864a3391506l8170626t1w339712730744-labbucket-pk5q3cfyfaoh'

# 定義存取 AWS Comprehend 的角色 ARN，用於授權訪問 Comprehend 服務的權限
database_access_role_arn = 'arn:aws:iam::339712730744:role/service-role/c133864a3391506l8170626t1w-ComprehendDataAccessRole-ox04SNmkCvJt'

# 定義存取 AWS Translate 的角色 ARN，用於授權訪問 Translate 服務的權限
translate_access_role_arn = 'arn:aws:iam::339712730744:role/c133864a3391506l8170626t1w3397127-TranslateDemoRole-MORbCAxdLUuo'


# 建立一個 AWS Transcribe 的客戶端，用於呼叫 AWS Transcribe 服務
transcribe_client = boto3.client("transcribe")

test.wav 範例檔案位於 **/s3** 資料夾中。

該文件包含音頻短語 `測試。你好，你好，你好。這是一個測試。測試，測試，測試。`

In [8]:
media_input_uri = f's3://{bucket}/lab71/transcribe-sample/test.wav'

### 檢查是否具備權限

In [9]:
import boto3

s3_client = boto3.client("s3")
try:
    s3_client.head_bucket(Bucket=bucket)
    print("Bucket exists and is accessible.")
except Exception as e:
    print(f"Error accessing bucket: {e}")

Bucket exists and is accessible.


首先，使用 test.wav 檔案作為輸入建立轉錄作業。請注意，您需要指定輸出位置。

In [11]:
# 建立轉錄作業
job_uuid = uuid.uuid1()
transcribe_job_name = f"transcribe-job-{job_uuid}"
transcribe_output_filename = 'transcribe_output.txt'

response = transcribe_client.start_transcription_job(
    TranscriptionJobName=transcribe_job_name,
    Media={'MediaFileUri': media_input_uri},
    MediaFormat='wav',
    LanguageCode='en-US',
    OutputBucketName=bucket,
    OutputKey=transcribe_output_filename
)

等待作業完成

In [12]:
job=None
while True:
    job = transcribe_client.get_transcription_job(TranscriptionJobName = transcribe_job_name)
    if job['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED','FAILED']:
        break
    print('.', end='')
    sleep(20)
        
print(job['TranscriptionJob']['TranscriptionJobStatus'])

COMPLETED


如果前一個儲存格的輸出是*COMPLETED*，則繼續。否則，請修正錯誤並重試前一個儲存格。

若要檢索輸出文件，請使用“get_transcription_job”方法呼叫的結果。

In [13]:
transcription_file = job['TranscriptionJob']['Transcript']['TranscriptFileUri']
print(transcription_file)

https://s3.us-east-1.amazonaws.com/c133864a3391506l8170626t1w339712730744-labbucket-pk5q3cfyfaoh/transcribe_output.txt


從 S3 下載文件

In [14]:
s3_client = boto3.client('s3')
with open(transcribe_output_filename, 'wb') as f:
    s3_client.download_fileobj(bucket, transcribe_output_filename, f)

開啟檔案並將內容讀入 JSON 物件

In [15]:
with open(transcribe_output_filename) as f:
  data = json.load(f)

In [16]:
data

{'jobName': 'transcribe-job-f9baf76a-9602-11ef-8cdf-3a328b8844c7',
 'accountId': '339712730744',
 'status': 'COMPLETED',
 'results': {'transcripts': [{'transcript': 'Test. Hello. Hello. Hello. This is a test test test test.'}],
  'items': [{'id': 0,
    'type': 'pronunciation',
    'alternatives': [{'confidence': '0.996', 'content': 'Test'}],
    'start_time': '0.009',
    'end_time': '0.56'},
   {'id': 1,
    'type': 'punctuation',
    'alternatives': [{'confidence': '0.0', 'content': '.'}]},
   {'id': 2,
    'type': 'pronunciation',
    'alternatives': [{'confidence': '0.997', 'content': 'Hello'}],
    'start_time': '0.66',
    'end_time': '1.19'},
   {'id': 3,
    'type': 'punctuation',
    'alternatives': [{'confidence': '0.0', 'content': '.'}]},
   {'id': 4,
    'type': 'pronunciation',
    'alternatives': [{'confidence': '0.996', 'content': 'Hello'}],
    'start_time': '1.2',
    'end_time': '1.7'},
   {'id': 5,
    'type': 'punctuation',
    'alternatives': [{'confidence': '0.0'

獲得實際的轉錄

In [17]:
data['results']['transcripts'][0]['transcript']

'Test. Hello. Hello. Hello. This is a test test test test.'

## 2. Amazon Translate example

本部分將使用 boto3 用戶端呼叫 `Amazon Translate` 並將文字檔案從英文轉換為西班牙文。

運行後可轉到 `Amazon Translate` 控制台查看翻譯。

有關作業的翻譯和詳細資訊位於 **批量翻譯** 部分。

包含翻譯的文字檔案將位於您的 S3 儲存桶中。該儲存桶還將有一個 **details** 資料夾，其中包含一個 JSON 文件，其中包含有關翻譯的詳細信息，例如源語言和目標語言。

First, create the translation job. The input and output locations are required.

Note that Amazon Translate can translate the same text into multiple target languages. In this example, you will use Spanish, for which the language code is `es`.

In [18]:
import uuid

translate_client = boto3.client(service_name='translate')

input_data = f's3://{bucket}/lab71/translate-sample'
output_data = f's3://{bucket}'

job_uuid = uuid.uuid1()
translate_job_name = f"translate-job-{job_uuid}"
translate_job_submission = translate_client.start_text_translation_job(
    JobName=translate_job_name,
    InputDataConfig={'S3Uri': input_data, 'ContentType':'text/plain'},
    OutputDataConfig={'S3Uri': output_data},
    DataAccessRoleArn=translate_access_role_arn,
    SourceLanguageCode='en',
    TargetLanguageCodes=['es']
)
translate_job_id = translate_job_submission['JobId']

使用上一個儲存格中的作業 ID 來取得狀態。等待作業完成。

請注意，該作業將需要幾分鐘才能完成。

本機第一次實作花了 `15` 分鐘

In [19]:
while True:
    translate_job = translate_client.describe_text_translation_job(JobId=translate_job_id)
    if translate_job['TextTranslationJobProperties']['JobStatus'] in ['COMPLETED','FAILED']:
        break
    sleep(20)
    print('.', end='')

print(translate_job['TextTranslationJobProperties']['JobStatus'])

......................................COMPLETED


輸出 *COMPLETED* 再繼續

輸出資料夾的格式是根據帳號和作業 ID 建立的。
以下儲存格使用此資訊建立一條路徑。

In [20]:
account_id = boto3.client('sts').get_caller_identity().get('Account')
translate_output_path = f'{account_id}-TranslateText-{translate_job_id}/'

Amazon Translate 輸出多個檔案。您對 .txt 檔案感興趣，其中包含翻譯結果。以下儲存格將下載 .txt 檔案。

In [21]:
s3_resource = boto3.resource('s3')
my_bucket = s3_resource.Bucket(bucket)

for my_bucket_object in my_bucket.objects.filter(Prefix=translate_output_path):
    file=my_bucket_object.key
    if file.endswith('txt'):
        file = file.lstrip(translate_output_path)
        file = file.lstrip('/')
        print(file)
        with open(file, 'wb') as f:
            s3_client.download_fileobj(bucket, my_bucket_object.key, f)

.test.txt


## 3. Amazon Polly example

本部分中使用 boto3 呼叫 Amazon Polly 並建立西班牙文文字檔案的發聲。

運行後打開 S3 儲存桶以查看輸出。

輸出是一個 .mp3 文件，其文件名是長字串。可開啟檔案並聽到 Lucia 的聲音說 `Prueba de prueba, este es una prueba`。


In [22]:
polly_client = boto3.client('polly')

itemname = 'lab71/polly-sample/es.test.txt'
obj = s3_resource.Object(bucket, itemname )
body = obj.get()['Body'].read().decode('utf-8')

response = polly_client.start_speech_synthesis_task(
    Engine='standard',
    OutputFormat='mp3',
    OutputS3BucketName=bucket,
    Text=body,
    VoiceId='Lucia'
) 


從回應中提取任務 ID。

In [24]:
task_id = response['SynthesisTask']['TaskId']
task_id

'67f557e5-92cd-4d59-a07a-5cddef857497'

使用此任務 ID 檢查作業是否已完成。

In [25]:
while True:
    polly_job = polly_client.get_speech_synthesis_task(TaskId=task_id)
    if polly_job['SynthesisTask']['TaskStatus'] in ['completed','failed']:
        break
    sleep(20)
    print('.', end='')

print(polly_job['SynthesisTask']['TaskStatus'])

completed


如果前一個單元格的輸出“已完成”，則繼續。否則，請修正錯誤並重試前一個儲存格。

若要下載結果，請執行下列儲存格。

In [26]:
s3_client = boto3.client('s3')
polly_output_filename = f'{task_id}.mp3'
with open(polly_output_filename, 'wb') as f:
    s3_client.download_fileobj(bucket, polly_output_filename, f)

## 4. Challenge exercise

您的挑戰是從具有英語音訊通道的影片建立翻譯後的音訊檔案。

您可以使用前三個範例中的程式碼作為解決方案的範本。

挑戰影片位於 S3 儲存桶的「lab71/challenge」資料夾中。影片檔名為sample.mp4。此文件也可在此筆記本實例的「/s3」資料夾中找到。